In [45]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from functools import reduce

In [46]:
tickers = ["AMJ","XLY","XLB","^GSPC","^DJI","^IXIC","^NYA","^RUT"]
start_date = "2018-08-19"
end_date = "2023-09-30"
etf_data = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    
    etf_data[ticker] = data

    
# ETF data frames
amj_df = etf_data["AMJ"]
xly_df = etf_data["XLY"]
xlb_df = etf_data["XLB"]

# Index data frames
gspc_df = etf_data["^GSPC"]
dji_df = etf_data["^DJI"]
ixic_df = etf_data["^IXIC"]
nya_df = etf_data["^NYA"]
rut_df = etf_data["^RUT"]

dji_df = dji_df.rename(columns={'Adj Close': 'DJI_Adj Close', 'Close': 'DJI_Close', 'High': 'DJI_High', 'Low': 'DJI_Low', 'Open': 'DJI_Open','Volume': 'DJI_Volume'})
gspc_df = gspc_df.rename(columns={'Adj Close': 'GSPC_Adj Close', 'Close': 'GSPC_Close', 'High': 'GSPC_High', 'Low': 'GSPC_Low', 'Open': 'GSPC_Open','Volume': 'GSPC_Volume'})
ixic_df = ixic_df.rename(columns={'Adj Close': 'IXIC_Adj Close', 'Close': 'IXIC_Close', 'High': 'IXIC_High', 'Low': 'IXIC_Low', 'Open': 'IXIC_Open','Volume': 'IXIC_Volume'})
nya_df = nya_df.rename(columns={'Adj Close': 'NYA_Adj Close', 'Close': 'NYA_Close', 'High': 'NYA_High', 'Low': 'NYA_Low', 'Open': 'NYA_Open','Volume': 'NYA_Volume'})
rut_df = rut_df.rename(columns={'Adj Close': 'RUT_Adj Close', 'Close': 'RUT_Close', 'High': 'RUT_High', 'Low': 'RUT_Low', 'Open': 'RUT_Open','Volume': 'RUT_Volume'})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [47]:
etf_data

{'AMJ':                  Open       High        Low      Close  Adj Close   Volume
 Date                                                                      
 2018-08-20  29.200001  29.549999  29.190001  29.510000  19.672087  1523300
 2018-08-21  29.510000  29.610001  29.230000  29.230000  19.485430  1050600
 2018-08-22  29.430000  29.559999  28.809999  29.520000  19.678753  1565700
 2018-08-23  29.410000  29.540001  29.320000  29.540001  19.692087   742500
 2018-08-24  29.469999  29.490000  29.260000  29.410000  19.605419  1288600
 ...               ...        ...        ...        ...        ...      ...
 2023-09-25  24.580000  24.870001  24.370001  24.719999  24.719999   710800
 2023-09-26  24.559999  24.600000  24.270000  24.299999  24.299999   181400
 2023-09-27  24.389999  24.660000  24.389999  24.600000  24.600000   384100
 2023-09-28  24.400000  24.879999  24.400000  24.870001  24.870001   604400
 2023-09-29  24.950001  24.950001  24.639999  24.700001  24.700001   275000
 
 [1

In [48]:
def analytical_inferences(etf_data):
    '''
    Function calculates all inferences possible like RSI, EMA, SMA. 
    Input: 1 data frame containing ETF data 
    Output: The data frame with all calculated values for the particular ETF.
    '''
    rsi_period = 14
    # Calculate RSI
    etf_data['RSI'] = ta.rsi(etf_data['Adj Close'], length=rsi_period)

    # Calculate overbought/oversold conditions
    etf_data['Overbought'] = (etf_data['RSI'] > 70).astype(int)
    etf_data['Oversold'] = (etf_data['RSI'] < 30).astype(int)
    
    # Calculate divergence between price and RSI
    etf_data['Price_RSI_Divergence'] = etf_data['Close'].diff() - etf_data['RSI'].diff()
    
    # Calculate rate of change of RSI
    etf_data['ROC_RSI'] = etf_data['RSI'].pct_change() * 100
    
    # Calculate RSI trend confirmation
    etf_data['RSI_Trend_Confirmation'] = (etf_data['RSI'] > etf_data['RSI'].shift(1)).astype(int)
    
    # Assuming 'Close' is the column containing closing prices
    etf_data['EMA'] = ta.ema(etf_data['Close'], length=14)  # Adjust the period as needed
    
    # Feature 1: EMA over a specific period
    # Already calculated and stored in 'EMA' column
    
    # Feature 2: Difference between current price and EMA
    etf_data['Price_EMA_Difference'] = etf_data['Close'] - etf_data['EMA']
    
    # Feature 3: Slope of EMA
    etf_data['Slope_EMA'] = ta.slope(etf_data['EMA'])
    
    # Feature 4: EMA convergence or divergence
    etf_data['EMA_Convergence'] = (etf_data['Close'] > etf_data['EMA']).astype(int)
    etf_data['EMA_Divergence'] = (etf_data['Close'] < etf_data['EMA']).astype(int)
    
    # Feature 5: Rate of change of EMA
    etf_data['ROC_EMA'] = etf_data['EMA'].pct_change() * 100
    
    # Assuming 'Close' is the column containing closing prices
    etf_data['SMA'] = ta.sma(etf_data['Close'], length=14)  # Adjust the period as needed
    
    # Feature 1: SMA over a specific period
    # Already calculated and stored in 'SMA' column
    
    # Feature 2: Difference between current price and SMA
    etf_data['Price_SMA_Difference'] = etf_data['Close'] - etf_data['SMA']
    
    # Feature 3: Slope of SMA
    etf_data['Slope_SMA'] = ta.slope(etf_data['SMA'])
    
    # Feature 4: SMA convergence or divergence
    etf_data['SMA_Convergence'] = (etf_data['Close'] > etf_data['SMA']).astype(int)
    etf_data['SMA_Divergence'] = (etf_data['Close'] < etf_data['SMA']).astype(int)
    
    # Feature 5: Rate of change of SMA
    etf_data['ROC_SMA'] = etf_data['SMA'].pct_change() * 100
    
    dmi = ta.adx(etf_data.High, etf_data.Low, etf_data.Close)
    etf_data['ADX']=dmi['ADX_14']
    etf_data['DMI+']=dmi['DMP_14']
    etf_data['DMI-']=dmi['DMN_14']
    # Calculate ADX trend strength
    etf_data['ADX_Trend_Strength'] = etf_data['ADX'].rolling(window=3).mean()  # Adjust the rolling window parameter
    
    # Calculate DI convergence or divergence
    etf_data['DI_Convergence_Divergence'] = etf_data['DMI+'] - etf_data['DMI-']  # Adjust the length parameter
    return etf_data

In [49]:
amj_df = analytical_inferences(amj_df)
xly_df = analytical_inferences(xly_df)
xlb_df = analytical_inferences(xlb_df)

In [50]:
amj_combined_df = pd.DataFrame(reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), [amj_df, dji_df, gspc_df, ixic_df, nya_df, rut_df]))
xly_combined_df = pd.DataFrame(reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), [xly_df, dji_df, gspc_df, ixic_df, nya_df, rut_df]))
xlb_combined_df = pd.DataFrame(reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), [xlb_df, dji_df, gspc_df, ixic_df, nya_df, rut_df]))

In [51]:
amj_combined_df

,Open,High,Low,Close,Adj Close,Volume,RSI,Overbought,Oversold,Price_RSI_Divergence,...,NYA_Low,NYA_Close,NYA_Adj Close,NYA_Volume,RUT_Open,RUT_High,RUT_Low,RUT_Close,RUT_Adj Close,RUT_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2018-08-20,29.200001,29.549999,29.190001,29.510000,19.672087,1523300,NaN,0,0,NaN,...,12929.429688,12965.099609,12965.099609,2757150000,1693.790039,1701.130005,1690.180054,1698.689941,1698.689941,2757150000
2018-08-21,29.510000,29.610001,29.230000,29.230000,19.485430,1050600,NaN,0,0,NaN,...,12993.950195,12996.759766,12996.759766,3174010000,1698.910034,1722.290039,1698.910034,1718.050049,1718.050049,3174010000
2018-08-22,29.430000,29.559999,28.809999,29.520000,19.678753,1565700,NaN,0,0,NaN,...,12983.019531,12990.509766,12990.509766,2701250000,1717.750000,1723.010010,1715.800049,1722.540039,1722.540039,2701250000
2018-08-23,29.410000,29.540001,29.320000,29.540001,19.692087,742500,NaN,0,0,NaN,...,12922.839844,12933.459961,12933.459961,2724450000,1722.300049,1723.410034,1712.920044,1717.050049,1717.050049,2724450000
2018-08-24,29.469999,29.490000,29.260000,29.410000,19.605419,1288600,NaN,0,0,NaN,...,12964.299805,12999.440430,12999.440430,2603010000,1718.359985,1726.969971,1718.359985,1725.670044,1725.670044,2603010000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,24.580000,24.870001,24.370001,24.719999,24.719999,710800,67.282011,0,0,-3.744703,...,15491.230469,15593.730469,15593.730469,3195650000,1768.670044,1788.030029,1766.540039,1784.239990,1784.239990,3195650000
2023-09-26,24.559999,24.600000,24.270000,24.299999,24.299999,181400,54.600185,0,0,12.261825,...,15371.750000,15381.580078,15381.580078,3472340000,1774.640015,1786.050049,1761.609985,1761.609985,1761.609985,3472340000
2023-09-27,24.389999,24.660000,24.389999,24.600000,24.600000,384100,60.349195,0,0,-5.449009,...,15288.450195,15394.690430,15394.690430,3875880000,1771.160034,1786.599976,1765.640015,1778.900024,1778.900024,3875880000
